In [2]:
%load_ext sql
%sql sqlite:///PS1.db

Problem Set 1 [100 points]
=======

### Deliverables:

Submit your queries (and only those) using the `submission_template.txt` file that is posted on Canvas. Follow the instructions on the file! Upload the file at Canvas (under PS1).


### Instructions / Notes:

* Run the top cell above to load the database `PS1.db` (make sure the database file, `PS1.db`, is in the same directory as this IPython notebook is running in)
* Some of the problems involve _changing_ this database (e.g. deleting rows)- you can always re-download `PS1.db` or make a copy if you want to start fresh!
* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that your final answer for each question is _in its own cell_ and _clearly indicated_**
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging- i.e. running for too long: To restart the SQL connection, you must restart the entire python kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute the sql connection cell at top
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql [SQL]` is for _multi line_ SQL queries
* _Have fun!_

Problem 1: Linear Algebra [25 points]
------------------------

Two random 3x3 ($N=3$) matrices have been provided in tables `A` and `B`, having the following schema:
> * `i INT`:   Row index
> * `j INT`:   Column index
> * `val INT`: Cell value

**Note: all of your answers below _must_ work for any _square_ matrix sizes, i.e. any value of $N$**.

Note how the matrices are represented- why do we choose this format?  Run the following queries to see the matrices in a nice format:

In [27]:
%sql SELECT group_concat(val, ' , ') AS "A" FROM A GROUP BY i;

 * sqlite:///PS1.db
Done.


A
"7 , 5 , 8"
"10 , 7 , 7"
"2 , 0 , 5"


In [28]:
%sql SELECT group_concat(val, ' , ') AS "B" FROM B GROUP BY i;

 * sqlite:///PS1.db
Done.


B
"9 , 6 , 10"
"7 , 6 , 9"
"1 , 1 , 7"


### Part (a): Matrix transpose [5 points]

_Transposing_ a matrix $A$ is the operation of switching its rows with its columns- written $A^T$.  For example, if we have:

$A=\begin{bmatrix}a & b & c\\ d & e & f\\ g & h & i\end{bmatrix}$

Then:

$A^T=\begin{bmatrix}a & d & g\\ b & e & h\\ c & f & i\end{bmatrix}$

Write a _single SQL query_ to get the matrix transpose $A^T$ (in the same format as $A$- output tuples should be of format `(i,j,val)` where `i` is row, `j` is column, and the output is ordered by row then column index)

Write your query here:

In [54]:
%sql SELECT A.j AS i,A.i AS j, val FROM A GROUP BY A.j, A.i;

 * sqlite:///PS1.db
Done.


i,j,val
0,0,7
0,1,10
0,2,2
1,0,5
1,1,7
1,2,0
2,0,8
2,1,7
2,2,5


### Part (b): Dot product I [5 points]

The _dot product_ of two vectors

$a = \begin{bmatrix}a_1 & a_2 & \dots & a_n\end{bmatrix}$

and

$b = \begin{bmatrix}b_1 & b_2 & \dots & b_n\end{bmatrix}$

is

$a\cdot b = \sum_{i=1}^n a_ib_i = a_1b_1 + a_2b_2 + \dots + a_nb_n$

Write a _single SQL query_ to take the dot product of the **third column of $A$** and the **second column of $B$.**

Write your query here:

In [55]:
%sql SELECT SUM(A.val*B.val) AS dotProduct FROM A, B WHERE A.j = 2 AND B.j = 1 AND A.i = B.i;


 * sqlite:///PS1.db
Done.


dotProduct
95


### Part (c): Matrix multiplication [10 points]

The product of a matrix $A$ (having dimensions $n\times m$) and a matrix $B$ (having dimensions $m\times p$) is the matrix $C$ (of dimension $n\times p$) having cell at row $i$ and column $j$ equal to:

$C_{ij} = \sum_{k=1}^m A_{ik}B_{kj}$

In other words, to multiply two matrices, get each cell of the resulting matrix $C$, $C_{ij}$, by taking the _dot product_ of the $i$th row of $A$ and the $j$th column of $B$.

Write a single SQL query to get the matrix product of $A$ and $B$ (in the same format as $A$ and $B$).

Write your query here:

In [56]:
%sql SELECT A.i, B.j, SUM(A.val*B.val) AS val FROM A,B WHERE B.i = A.j GROUP BY A.i, B.j;

 * sqlite:///PS1.db
Done.


i,j,val
0,0,106
0,1,80
0,2,171
1,0,146
1,1,109
1,2,212
2,0,23
2,1,17
2,2,55


### Part (d): Matrix power [5 points]

The power $A^n$ of a matrix $A$ is defined as the matrix product of $n$ copies of $A$. 

Write a _single SQL query_ that computes the **third power** of matrix $A$ (in the same format as $A$), in other words, $A^3 = A \cdot A \cdot A$.

Write your query here:

In [57]:
%%sql
SELECT A.i, B.j, SUM(A.val*B.val) AS val
FROM A,(SELECT A1.i, A2.j, SUM(A1.val*A2.val) AS val 
		FROM A A1,A A2 
        WHERE A2.i = A1.j 
        GROUP BY A1.i, A2.j) AS B
WHERE B.i = A.j 
GROUP BY A.i, B.j;

 * sqlite:///PS1.db
Done.


i,j,val
0,0,1767
0,1,1065
0,2,2065
1,0,2396
1,1,1463
1,2,2745
2,0,350
2,1,190
2,2,467


Problem 2: The Sales Database [35 points]
----------------------------------------------

We've prepared and loaded a dataset related to sales data from a company. The dataset has the following schema:

> `Holidays (WeekDate, IsHoliday)`

> `Stores (Store, Type, Size)`

> `TemporalData(Store, WeekDate, Temperature, FuelPrice, CPI, UnemploymentRate)`

> `Sales (Store, Dept, WeekDate, WeeklySales)`

Before you start writing queries on the database, find the schema and the constraints (keys, foreign keys). 

### Part (a): Sales during Holidays [5 points]

Using a single SQL query, find the stores with the largest and smallest overall sales during holiday weeks. Further requirements:
* Use the `WITH` clause before the main body of the query to compute a subquery if necessary.
* Return a relation with schema `(Store, AllSales)`.

Write your query here:

In [60]:
%%sql
WITH t AS (SELECT S.store AS Store, SUM(S.weeklysales) AS ALLSales
           FROM sales S, holidays H
           WHERE S.weekdate = H.weekdate
           AND H.isholiday = 'TRUE'
           GROUP BY Store)

SELECT * 
FROM t
WHERE ALLSales IN(SELECT MAX(ALLSales) FROM t)
OR ALLSales IN(SELECT MIN(ALLSales) FROM t);


 * sqlite:///PS1.db
Done.


Store,ALLSales
20,22490350.81000001
33,2625945.1900000004


### Part (b): When Holidays do not help Sales [10 points]

Using a single SQL query, compute the **number** of non-holiday weeks that had larger sales than the overall average sales during holiday weeks. Further requirements:
* Use the `WITH` clause before the main body of the query to compute a subquery if necessary.
* Return a relation with schema `(NumNonHolidays)`.

Write your query here:

In [61]:
%%sql
WITH h AS (SELECT SUM(S.weeklysales)/COUNT(DISTINCT S.weekdate) as avg
           FROM sales S, holidays H
           WHERE S.weekdate = H.weekdate
           AND H.isholiday = 'TRUE'),
n AS (SELECT DISTINCT (S.weekdate) AS W, SUM(S.weeklysales) as WSum
      FROM sales S, holidays H
      WHERE S.weekdate = H.weekdate
      AND H.isholiday = 'FALSE'
      GROUP BY W)

SELECT COUNT(n.W) AS NumNonHolidays
FROM n,h
WHERE n.WSum > h.avg;

 * sqlite:///PS1.db
Done.


NumNonHolidays
8


### Part (c): Total Sales [5 points]

Using a _single SQL query_, compute the total sales during summer (months 6,7and 8) for each type of store. Further requirements:
* Return a relation with schema `(type, TotalSales)`.

*Hint:* SQLite3 does not support native operations on the DATE datatype. To create a workaround, you can use the `LIKE` predicate and the string concatenation operator (||). You can also use the substring operator that SQLite3 supports (`substr`).

Write your query here:

In [62]:
%%sql
SELECT ST.type, SUM(S.weeklysales) AS TotalSales
FROM sales S,stores ST
WHERE (strftime('%m',S.weekdate) = '06' OR strftime('%m',S.weekdate) = '07' OR strftime('%m',S.weekdate) = '08')
       AND S.store = ST.store
GROUP BY ST.type;


 * sqlite:///PS1.db
Done.


type,TotalSales
A,1211554899.8500097
B,561610722.3999932
C,112555450.65999933


### Part (d): Computing Correlations [15 points]

The goal of this exercise is to find out whether each one of the 4 numeric attributes in `TemporalData` is
positively or negatively correlated with sales.

For our purposes, the intuitive notion of correlation is defined using a
standard statistical quantity known as the *Pearson correlation coefficient*.
Given two numeric random variables $X$ and $Y$, it is defined as follows:

$$\rho_{X,Y} = \frac{E[XY] - E[X]E[Y]}{\sqrt{E[X^2] - E[X]^2} \sqrt{E[Y^2] - E[Y]^2}}$$

On a given sample of data with $n$ examples each for $X$ and $Y$ (label them
$x_i$ and $y_i$ respectively for $i = 1 \dots n$), it is estimated as follows:

\begin{align*}
\rho_{X,Y} = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2} \sqrt{\sum_{i=1}^n (y_i - \bar{y})^2}}
\end{align*}
In the above, $\bar{x}$ and $\bar{y}$ are the sample means, i.e.,
$\bar{x} = \frac{1}{n}\sum_{i=1}^n x_i$, and $\bar{x} = \frac{1}{n}\sum_{i=1}^n y_i$.

Further requirements:
* Each example in the sample consists of a pair (Store, WeekDate). This means that Sales data must be summed across all departments before the correlation is computed. 
* Return a relation with schema `(AttributeName VARCHAR(20), CorrelationSign Integer)`. 
* The values of AttributeName can be hardcoded string literals, but the values
of `CorrelationSign` must be computed automatically using SQL queries over
the given database instance.
* You can use multiple SQL statements to compute the result. It might be of help to create and update your own tables.

Write your query here:

In [63]:
%%sql
WITH salesData AS(SELECT store,weekdate,SUM(weeklysales) AS ws 
                  FROM sales
                  GROUP BY store,weekdate),
average AS (SELECT AVG(temperature) AS avgT,AVG(cpi) AS avgC,AVG(unemploymentrate) AS avgU,AVG(fuelprice) AS avgF
             FROM temporaldata),
avgX AS(SELECT AVG(ws) AS x
        FROM salesData)

SELECT  cast('temperature' AS VARCHAR(20)) AS AttributeName,cast(SUM(T.temperature - A.avgT)*(S.ws - avgX.x) AS INTEGER)/ ABS(cast(SUM(T.temperature - A.avgT)*(S.ws - avgX.x) AS INTEGER))AS CorrelationSign
FROM temporaldata T, average A,salesData S, avgX

UNION
   
SELECT  cast('fuelprice' AS VARCHAR(20)) AS AttributeName,cast(SUM(T.fuelprice - A.avgF)*(S.ws - avgX.x) AS INTEGER)/ ABS(cast(SUM(T.fuelprice - A.avgF)*(S.ws - avgX.x) AS INTEGER)) AS CorrelationSign
FROM temporaldata T, average A,salesData S, avgX

UNION

SELECT  cast('cpi' AS VARCHAR(20)) AS AttributeName,cast(SUM(T.cpi - A.avgC)*(S.ws - avgX.x) AS INTEGER)/ ABS(cast(SUM(T.cpi - A.avgC)*(S.ws - avgX.x) AS INTEGER)) AS CorrelationSign
FROM temporaldata T, average A,salesData S, avgX

UNION 

SELECT  cast('unemploymentrate' AS VARCHAR(20)) AS AttributeName,cast(SUM(T.unemploymentrate - A.avgU)*(S.ws - avgX.x) AS INTEGER)/ ABS(cast(SUM(T.unemploymentrate - A.avgU)*(S.ws - avgX.x) AS INTEGER)) AS CorrelationSign
FROM temporaldata T, average A,salesData S, avgX;


 * sqlite:///PS1.db
Done.


AttributeName,CorrelationSign
cpi,-1
fuelprice,1
temperature,-1
unemploymentrate,-1


In [12]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

"\nExpected output below- don't re-evaluate this cell!\n\nNOTE: A valid answer must work for ALL inputs of the given type,\nnot just this example.  I.e. do not hardcode around this answer / etc!\n"

Problem 3: The Traveling SQL Server Salesman Problem [40 points]
--------------------------------------------------

SQL Server salespeople are lucky as far as traveling salespeople go- they only have to sell one or two big enterprise contracts, at one or two offices in Wisconsin, in order to make their monthly quota!

Answer the following questions using the table of streets connecting company office buildings.

**Note that for convenience all streets are included _twice_, as $A \rightarrow B$ and $B \rightarrow A$.  This should make some parts of the problem easier, but remember to take it into account!**

In [13]:
%sql SELECT * FROM streets LIMIT 4;

 * sqlite:///PS1.db
Done.


id,direction,A,B,d
0,F,UW-Madison,DooHickey Collective,7
0,R,DooHickey Collective,UW-Madison,7
1,F,DooHickey Collective,Gizmo Corp,2
1,R,Gizmo Corp,DooHickey Collective,2


### Part (a): One-hop, two-hop, three-hop... [10 points]

Our salesperson has stopped at UW-Madison, to steal some cool new RDBMS technology from CS564-ers, and now wants to go sell it to a company _within 10 miles of UW-Madison and _passing through no more than 3 distinct streets_.  Write a single query, not using `WITH` (see later on), to find all such companies.

Your query should return the schema `(company, distance)` where distance is cumulative from UW-Madison.

Write your query here:

In [64]:
%%sql
SELECT DISTINCT st.B AS company, st.d AS distance
 FROM streets st 
 WHERE st.A = 'UW-Madison' 
 AND st.d <= 10

UNION

SELECT DISTINCT st2.B AS company, (st1.d+st2.d) AS distance
 FROM streets st1, streets st2 
 WHERE st1.A = 'UW-Madison' 
 AND (st1.d+st2.d)<= 10
 AND st1.B = st2.A 
 AND st2.B != 'UW-Madison'

UNION

SELECT DISTINCT st3.B AS company, (st1.d+st2.d+st3.d) AS distance
 FROM streets st1, streets st2,streets st3
 WHERE st1.A = 'UW-Madison' 
 AND (st1.d+st2.d+st3.d)<= 10
 AND st1.B = st2.A AND st2.B != st1.A 
 AND st2.B = st3.A AND st3.B != st2.A 
 AND st3.B != 'UW-Madison' ;

 * sqlite:///PS1.db
Done.


company,distance
DooHickey Collective,7
DooHickey Corp,9
Gadget Collective,9
Gadget Corp,6
Gizmo Corp,9
Widget Industries,10


In [15]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

"\nExpected output below- don't re-evaluate this cell!\n\nNOTE: A valid answer must work for ALL inputs of the given type,\nnot just this example.  I.e. do not hardcode around this answer / etc!\n"

### Part (b): A stop at the Farm [10 points]

Now, our salesperson is out in the field, and wants to see all routes- and their distances- which will take him/her from a company $A$ to a company $B$, with the following constraints:
* The route must _pass through UW-Madison (in order to pick up new RDBMS tech to sell!)
* $A$ and $B$ must _each individually_ be _within 3 hops of UW-Madison
* $A$ and $B$ must be different companies
* _The total distance must be $<= 15$_
* Do not use `WITH`
* If you return a path $A \rightarrow B$, _also include $B \rightarrow A$ in your answer_

In order to make your answer a bit cleaner, you may split into two queries, one of which creates a `VIEW`.  A view is a virtual table based on the output set of a SQL query.  A view can be used just like a normal table- the only difference under the hood is that the DBMS re-evaluates the query used to generate it each time a view is queried by a user (thus the data is always up-to date!)

Here's a simple example of a view:

In [3]:
%%sql 
DROP VIEW IF EXISTS short_streets;
CREATE VIEW short_streets AS 
SELECT A, B, d FROM streets WHERE d < 3;
SELECT * FROM short_streets LIMIT 3;

 * sqlite:///PS1.db
Done.
Done.
Done.


A,B,d
DooHickey Collective,Gizmo Corp,2
Gizmo Corp,DooHickey Collective,2
Gizmo Corp,Widget Industries,1


Write your query or queries here:

In [65]:
%%sql 
DROP VIEW IF EXISTS table1;
CREATE VIEW table1 AS 

SELECT st.A AS A, st.B AS B, st.d AS d
FROM streets st 
WHERE st.A = 'UW-Madison' 
OR st.B = 'UW-Madison'
    
UNION

SELECT st1.A AS A, st2.B AS B, (st1.d + st2.d) AS d
FROM streets st1,streets st2
WHERE (st1.A = 'UW-Madison' OR st2.B = 'UW-Madison')
AND st1.A != st2.B
AND st1.B = st2.A

UNION 

SELECT st1.A AS A, st3.B AS B, (st1.d + st2.d + st3.d) AS d
FROM streets st1,streets st2,streets st3
WHERE (st1.A = 'UW-Madison' OR st3.B = 'UW-Madison')
AND st1.B = st2.A
AND st1.A != st2.B
AND st2.B = st3.A
AND st2.A != st3.B
AND st1.A != st3.B;


SELECT t1.A AS companyA, t2.B AS companyB, MIN(t1.d+t2.d) AS distance
FROM table1 t1, table1 t2
WHERE t1.B = 'UW-Madison' AND t2.A = 'UW-Madison' AND t1.A != t2.B
GROUP BY t1.A, t2.B
HAVING MIN(t1.d + t2.d) <= 15;
 





 * sqlite:///PS1.db
Done.
Done.
Done.


companyA,companyB,distance
DooHickey Collective,Gadget Corp,13
DooHickey Corp,Gadget Corp,15
Gadget Collective,Gadget Corp,15
Gadget Corp,DooHickey Collective,13
Gadget Corp,DooHickey Corp,15
Gadget Corp,Gadget Collective,15
Gadget Corp,Gizmo Corp,15
Gizmo Corp,Gadget Corp,15


In [18]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

"\nExpected output below- don't re-evaluate this cell!\n\nNOTE: A valid answer must work for ALL inputs of the given type,\nnot just this example.  I.e. do not hardcode around this answer / etc!\n"

### Part (c): Ensuring acyclicity [10 points]

You may have noticed at this point that the network, or **_graph_**, of streets in our `streets` table seems like it might be a **tree**.

Recall that a **_tree_** is an undirected graph where each node has exactly one parent (or, is the root, and has none), but may have multiple children.  A slightly more formal definition of a tree is as follows: 

> An undirected graph $T$ is a **_tree_** if it is **connected**- meaning that there is a path between every pair of nodes- and has no **cycles** (informally, closed loops)

Suppose that we guarantee the following about the graph of companies & streets determined by the `streets` table:
* It is _connected_
* It has no cycles of length $> 3$

Write a _single SQL query_ which, if our graph is not a tree (i.e. if this isn't a [shaggy dog problem](https://en.wikipedia.org/wiki/Shaggy_dog_story)), **turns it into a tree** by deleting exactly _one_ street (*= two entries in our table!*).  Write your query here:

In [66]:
%%sql
DELETE 
FROM streets
WHERE id IN(SELECT st1.id 
               FROM streets st1,streets st2,streets st3 
               WHERE st1.B = st2.A AND st2.B = st3.A AND st3.B = st1.A 
               LIMIT 1);


 * sqlite:///PS1.db
0 rows affected.


[]

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (d): The longest path [10 points]

In this part, we want to find the distance of the _longest path between any two companies_.

Note that you should probably have done Part (c) first so that the graph of streets is a _tree_- this will make it much easier to work with!

If you've done the other parts above, you might be skeptical that SQL can find paths of arbitrary lengths (which is what we need to do for this problem); how can we do this?

There are some non-standard SQL functions- i.e. not universally supported by all SQL DBMSs- which are often incredibly useful.  One of these is the `WITH RECURSIVE` clause, supported by SQLite.

A `WITH` clause lets you define what is essentially a view within a clause, mainly to clean up your SQL code.  A trivial example, to illustrate `WITH`:

In [21]:
%%sql
WITH companies(name) AS (
    SELECT DISTINCT A FROM streets)
SELECT * 
FROM companies 
WHERE name LIKE '%Gizmo%';

 * sqlite:///PS1.db
Done.


name
Gizmo Corp
GizmoWorks
Gizmo Industries
Gizmo Collective
GizmoCo


There is also a recursive variant, `WITH RECURSIVE`.  `WITH RECURSIVE` allows you to define a view just as above, except the table can be defined recursively.  A `WITH RECURSIVE` clause must be of the following form:

```
WITH RECURSIVE 
    R(...) AS (
        SELECT ... 
        UNION [ALL] 
        SELECT ... FROM R, ...
    )
...
```

`R` is the _recursive table_.  The `AS` clause contains two `SELECT` statements, conjoined by a `UNION` or `UNION ALL`; the first `SELECT` statement provides the initial / base case values, and the second or _recursive_ `SELECT` statement must include the recursive table in its `FROM` clause.

Basically, the recursive `SELECT` statement continues executing on the tuple _most recently inserted into `R`_, inserting output rows back into `R`, and proceeding recursively in this way, until it no longer outputs any rows, and then stops.  See the [SQLite documentation](https://www.sqlite.org/lang_with.html) for more detail.

The following example computes $5! = 5*4*3*2*1$ using `WITH RECURSIVE`:

In [22]:
%%sql
WITH RECURSIVE
    factorials(n,x) AS (
        SELECT 1, 1
        UNION
        SELECT n+1, (n+1)*x FROM factorials WHERE n < 5)
SELECT x FROM factorials WHERE n = 5;

 * sqlite:///PS1.db
Done.


x
120


In this example:

1. First, `(1,1)` is inserted into the table `factorials` (the base case).
2. Next, this tuple is returned by the recursive select, as `(n, x)`, and we insert the result back into `factorials`: `(1+1, (1+1)*1) = (2,2)`
3. Next, we do the same with the last tuple inserted into `factorials`- `(2,2)`- and insert `(2+1, (2+1)*2) = (3,6)`
4. And again: get `(3,6)` from `factorials` and insert `(3+1, (3+1)*6) = (4,24)` back in
5. And again: `(4,24)` -> `(4+1, (4+1)*24) = (5,120)`
6. Now the last tuple inserted into `factorials` is `(5, 120)`, which fails the `WHERE n < 5` clause, and thus our recursive select returns an empty set, and our `WITH RECURSIVE` statement concludes
7. Finally, now that our `WITH RECURSIVE` clause has concluded, we move on to the `SELECT x FROM factorials WHERE n=5` clause, which gets us our answer!

#### Now, your turn!

Write a single SQL query that uses `WITH RECURSIVE` to find the furthest (by distance) pair of companies that still have a path of streets connecting them.  Your query should return `(A, B, distance)`.

Write your query here:

In [67]:
%%sql
WITH RECURSIVE
    longestPath(a,b,pb,d) AS (
        SELECT A,B,A,d FROM streets
        UNION 
        SELECT lp.a, st.B, st.A, lp.d + st.d
        FROM longestPath lp, streets st
        WHERE lp.b = st.A AND st.B != lp.a AND st.B != lp.pb)
SELECT a AS A, b AS B, MAX(d) AS distance FROM longestPath;


 * sqlite:///PS1.db
Done.


A,B,distance
GadgetWorks,ThingWorks,63


In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Note on alternate output

**NOTE:** The **_distance_** of the longest path could be **49 _OR_ 63** depending on which street you deleted in Part (c)!